In [4]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
import numpy as np
import constantes as cons

lista=['GTCUSDT',
 'API3USDT',
 'LTCUSDT',
 '1000SHIBUSDT',
 'MKRUSDT',
 'MATICUSDT',
 'HIGHUSDT',
 '1000PEPEUSDT',
 'STMXUSDT',
 'APEUSDT',
 'TOMOUSDT',
 'BNBUSDT',
 'SUIUSDT',
 'CRVUSDT',
 'LINKUSDT',
 'GALAUSDT',
 'WLDUSDT',
 'HBARUSDT',
 'SOLUSDT',
 'XLMUSDT',
 'OPUSDT',
 'XRPUSDT',
 'COMPUSDT',
 'BCHUSDT',
 'DOGEUSDT',
 'YGGUSDT']
lista_filtrada = []

#lista=md.lista_de_monedas ()
lista=['SRMUSDT']


for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        data = md.estrategia_triangulos(symbol, tp_flag = True, print_lines_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = True, porcentajeentrada=100)
        if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


IMXUSDT - Return [%]: -0.45% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
FLMUSDT - Return [%]: 0.07% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
ENJUSDT - Return [%]: 0.02% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
SFPUSDT - Return [%]: -0.16% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:460: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=balance)


CFXUSDT - Return [%]: -0.12% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
OCEANUSDT - Return [%]: -0.02% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:460: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=balance)


LRCUSDT - Return [%]: 0.0% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
XEMUSDT - Return [%]: 0.02% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
IOTAUSDT - Return [%]: -0.08% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
STMXUSDT - Return [%]: 0.06% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
BNXUSDT - Return [%]: -0.07% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
AUDIOUSDT - Return [%]: -0.11% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
JOEUSDT - Return [%]: 0.01% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
C98USDT - Return [%]: -0.06% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
XLMUSDT - Return [%]: -0.04% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
RAYUSDT - Return [%]: -0.04% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
GMTUSDT - Return [%]: -0.08% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
ZRXUSDT - Return [%]: 0.0% - # Trades: 1 - Profit Factor: nan - Win Rat

In [ ]:
md.dibuja_patrones_triangulos (data,990) 